In [11]:
import pandas as pd
import os

In [14]:
# Parameters
config_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/configs/config.yaml"
path_alias_elec = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/customers/elec/alias_clientes_elec.xlsx"
path_raw_data_elec = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/invoice_comp/elec/raw_data_elec"
db_elec_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/facturas_elec.db"
db_elec_manual_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/facturas_elec_manual.pkl"
path_fras_elec_xml_clean = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/invoice_comp/elec/xml_folder_clean"


In [15]:
from config.config_loader import Config

config = Config(config_file=config_path)

In [16]:
from invoice_comp.check_utility import data_extraction
from utils.utils_var import insert_alias

In [17]:
df_alias = pd.read_excel(path_alias_elec)

In [18]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(f'sqlite:///{db_elec_path}', echo=True)

In [19]:
try:
    existing_data = pd.read_sql(('SELECT * FROM facturas'), con=engine)
    compiled_fras = existing_data['num_factura'].tolist()
except Exception as e:
    print(f"Error occurred or table does not exist: {e}")
    existing_data = pd.DataFrame()  
    compiled_fras = []

2025-08-16 12:49:52,154 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-16 12:49:52,158 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM facturas")
2025-08-16 12:49:52,158 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-16 12:49:52,159 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM facturas")
2025-08-16 12:49:52,159 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-16 12:49:52,160 INFO sqlalchemy.engine.Engine SELECT * FROM facturas
2025-08-16 12:49:52,160 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-16 12:49:52,172 INFO sqlalchemy.engine.Engine ROLLBACK


In [20]:
df = pd.DataFrame()

facturas_list = os.listdir(os.path.abspath(f'{path_fras_elec_xml_clean}'))
facturas = []

for fra in ((facturas_list)):
    
    if (("xml" in fra or "XML" in fra) and (fra[:-4] not in compiled_fras)):

        print (f'Factura-e: {fra}')
        path_factura = os.path.abspath(f'{path_fras_elec_xml_clean}/{fra}')
        conceptos_factura = data_extraction(path_factura)
        facturas.append(conceptos_factura)
        print()
        facturas_df = pd.DataFrame(facturas)
        df = pd.concat([df, facturas_df], ignore_index=True)

if not df.empty:
    df.drop_duplicates(subset='num_factura', inplace=True)
    df = df[~df['num_factura'].str.contains('VARIOS', case=False, na=False)]
    df['cliente'] = df.apply(lambda row: insert_alias(row['cups'], df_alias)[0], axis=1)
    df['alias'] = df.apply(lambda row: insert_alias(row['cups'], df_alias)[1], axis=1)
else:
    print("No new invoices found.")

No new invoices found.


In [21]:
if not len(df) == 0: 
    for column in ['inicio_periodo', 'fin_periodo', 'issue_date']:
        df[column] = pd.to_datetime(df[column])
        # df[column] = df[column].dt.date
    potencia_df = pd.DataFrame(df['potencia'].tolist(), index=df.index)
    potencia_df.columns = [f'potencia_p{i+1}' for i in range(potencia_df.shape[1])]
    energia_df = pd.DataFrame(df['energia'].tolist(), index=df.index)
    energia_df.columns = [f'energia_p{i+1}' for i in range(energia_df.shape[1])]
    maximetros_df = pd.DataFrame(df['maximetros'].tolist(), index=df.index)
    maximetros_df.columns = [f'maximetros_p{i+1}' for i in range(maximetros_df.shape[1])]
    df_facturas = pd.concat([df, potencia_df, energia_df, maximetros_df], axis=1)
    df_facturas = df_facturas.drop(columns=['potencia', 'energia', 'maximetros'])
    df_facturas = df_facturas.drop_duplicates(subset='num_factura')
    df_facturas['year'] = df_facturas['inicio_periodo'].dt.year
    df_facturas['month'] = df_facturas['inicio_periodo'].dt.month
    # df_expanded = df_expanded.groupby(['year','month', 'cups']).sum(numeric_only=True).reset_index()
    # df_expanded = df_expanded.groupby(['year','month']).sum(numeric_only=True).reset_index()
    df_facturas = df_facturas.round(2)
    # print(df_facturas.tail())

    columns = list(df_facturas.columns[5:7]) + list(df_facturas.columns[9:12]) + (list(df_facturas.columns[14:32]))
    print(df_facturas.columns[8])

    def enforce_negative(df: pd.DataFrame, column_list):
        # Select rows where 'num_factura' starts with 'FELEC'
        mask_felec = df['num_factura'].str.contains('ELE', na=False)

        # Within those rows, find rows with negative values in specified columns
        mask_negative = (df.loc[mask_felec, column_list] < 0).any(axis=1)

        # Get the indices of rows matching both conditions
        indices_to_modify = df.loc[mask_felec].index[mask_negative]

        # Enforce negative values
        df.loc[indices_to_modify, column_list] = df.loc[indices_to_modify, column_list].abs() * -1

        return df

    df_facturas = enforce_negative(df_facturas, columns)
else:
    df_facturas = existing_data


In [22]:
if not existing_data.empty:
    new_rows = df_facturas[~df_facturas['num_factura'].isin(existing_data['num_factura'])]
else:
    new_rows = df_facturas
if not new_rows.empty:
    new_rows.to_sql('facturas', con=engine, if_exists='append', index=False)
    print(f"{len(new_rows)} new rows have been added to the database.")
else:
    print("No new rows to add; all rows already exist in the database.")


No new rows to add; all rows already exist in the database.


In [23]:
df_facturas = pd.read_sql(('SELECT * FROM facturas'), con=engine)

2025-08-16 12:49:52,229 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-16 12:49:52,231 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM facturas")
2025-08-16 12:49:52,231 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-16 12:49:52,231 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM facturas")
2025-08-16 12:49:52,231 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-16 12:49:52,232 INFO sqlalchemy.engine.Engine SELECT * FROM facturas
2025-08-16 12:49:52,232 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-16 12:49:52,275 INFO sqlalchemy.engine.Engine ROLLBACK


In [24]:
df_facturas.sort_values(by='issue_date', ascending=False).head()

,num_factura,inicio_periodo,fin_periodo,issue_date,cups,coste_potencia,coste_energia,excesos_potencia,dto_electrointensivo,coste_reactiva,...,energia_p5,energia_p6,maximetros_p1,maximetros_p2,maximetros_p3,maximetros_p4,maximetros_p5,maximetros_p6,year,month
2461,FELEC_2500439872,2025-05-01 00:00:00.000000,2025-05-31 00:00:00.000000,2025-06-25 00:00:00.000000,ES0021000004106053EK,635.03,1361.47,0.0,0.0,0.0,...,4321.00,8554.00,0.0,0.0,0.0,126.0,114.0,92.0,2025,5
2453,FELEC_2500439118,2025-05-01 00:00:00.000000,2025-05-31 00:00:00.000000,2025-06-24 00:00:00.000000,ES0021000004832397RW,52.93,59.54,0.0,0.0,0.0,...,315.00,128.00,0.0,0.0,0.0,14.0,9.0,1.0,2025,5
2447,FELEC_2500439108,2025-05-01 00:00:00.000000,2025-05-31 00:00:00.000000,2025-06-24 00:00:00.000000,ES0031406105718001HD,424.31,1588.04,0.0,0.0,0.0,...,5832.43,5402.05,0.0,0.0,0.0,104.0,96.0,76.0,2025,5
2460,FELEC_2500439127,2025-05-19 00:00:00.000000,2025-06-16 00:00:00.000000,2025-06-24 00:00:00.000000,ES0031405525900001NA,7.57,7.87,0.0,0.0,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2025,5
2459,FELEC_2500439126,2025-05-02 00:00:00.000000,2025-06-01 00:00:00.000000,2025-06-24 00:00:00.000000,ES0022000007202761WE,32.51,37.16,0.0,0.0,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2025,5
